In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score  #交叉验证评分
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error #均方根误差
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
# 导入数据，路径中要么用\\或/或者在路径前加r
dataset = pd.read_csv('../data/train_chuli.csv')

In [50]:
# 输出数据预览
print(dataset.head())

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  \
0            1         0       3    1  22.0      1      0   7.2500   
1            2         1       1    0  38.0      1      0  71.2833   
2            3         1       3    0  26.0      0      0   7.9250   
3            4         1       1    0  35.0      1      0  53.1000   
4            5         0       3    1  35.0      0      0   8.0500   

   family_size  Embarked_C  Embarked_Q  Embarked_S  
0            2           0           0           1  
1            2           1           0           0  
2            1           0           0           1  
3            2           0           0           1  
4            1           0           0           1  


In [51]:
from sklearn.model_selection import train_test_split #划分数据集
# 准备训练数据
x = dataset.drop(['Survived'], axis = 1)
y = dataset['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0) 
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(623, 11) (268, 11) (623,) (268,)


In [52]:
from matplotlib.cm import rainbow 
from xgboost import XGBClassifier
#决策树
rf_scores = []

rf_classifier = XGBClassifier(max_depth=5, learning_rate=0.05, n_estimators=2000, colsample_bytree=0.1)
rf_classifier.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=2000,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

XGBClassifier
- learning_rate：学习率，控制每个基础模型的权重更新步长。较小的学习率可以帮助防止过拟合，但也可能需要更多的迭代次数才能收敛。默认值为 0.1。
- n_estimators：基础模型的数量，也就是迭代次数。默认值为 100。
- max_depth：每个基础模型的最大深度。较深的树可以更好地拟合复杂的数据集，但也更容易过拟合。默认值为 3。
- min_child_weight：每个叶子节点所需的最小权重。较小的值可以帮助模型更好地适应噪声数据，但也可能导致过拟合。默认值为 1。
- subsample：每个基础模型使用的样本比例。默认值为 1.0，表示使用所有的样本。较小的值可以帮助防止过拟合，但也可能导致欠拟合。
- colsample_bytree：每个基础模型使用的特征比例。默认值为 1.0，表示使用所有的特征。较小的值可以帮助防止过拟合，但也可能导致欠拟合。
- gamma：每个基础模型的损失函数减少多少才会进行拆分。默认值为 0，表示不进行拆分。较大的值可以帮助防止过拟合，但也可能导致欠拟合。
- reg_alpha：L1 正则化系数。默认值为 0，表示不进行正则化。
- reg_lambda：L2 正则化系数。默认值为 1，表示使用默认的 L2 正则化。
- scale_pos_weight：正样本权重，在不平衡分类问题中用于平衡正负样本比例。默认值为 1，表示不使用正样本权重。
- objective：分类问题的损失函数。默认值为 'binary:logistic'，表示使用二分类逻辑斯蒂回归损失函数。
- eval_metric：用于度量模型性能的指标。默认为 'error'，表示错误率。
- random_state：随机数生成器的种子，用于复现结果。

In [53]:
#评价指标
import numpy as np

# 计算混淆矩阵
def compute_confusion_matrix(precited,expected):
    part = precited ^ expected             # 对结果进行分类，亦或使得判断正确的为0,判断错误的为1
    pcount = np.bincount(part)             # 分类结果统计，pcount[0]为0的个数，pcount[1]为1的个数
    tp_list = list(precited & expected)    # 将TP的计算结果转换为list
    fp_list = list(precited & ~expected)   # 将FP的计算结果转换为list
    tp = tp_list.count(1)                  # 统计TP的个数
    fp = fp_list.count(1)                  # 统计FP的个数
    tn = pcount[0] - tp                    # 统计TN的个数
    fn = pcount[1] - fp                    # 统计FN的个数
    return tp, fp, tn, fn

# 计算常用指标
def compute_indexes(tp, fp, tn, fn):
    accuracy = (tp+tn) / (tp+tn+fp+fn)     # 准确率
    precision = tp / (tp+fp)               # 精确率
    recall = tp / (tp+fn)                  # 召回率
    F1 = (2*precision*recall) / (precision+recall)    # F1
    return accuracy, precision, recall, F1


In [54]:
rfc_score = rf_classifier.score(x_test, y_test)#测试集准确率,j即accuracy
expected = y_test.astype('int32')#预期值
# print(type(expected))
# print(expected)
rfc_pre=rf_classifier.predict(x_test)
precited = rfc_pre.astype('int32')#模型预测值
# print(type(precited))
tp, fp, tn, fn = compute_confusion_matrix(precited, expected)
accuracy, precision, recall, F1 = compute_indexes(tp, fp, tn, fn)
rfc_pre = rf_classifier.predict(x_test)
rfc_auc = metrics.roc_auc_score(y_test, rfc_pre)
rfc_rmse = np.sqrt(mean_squared_error(y_test, rfc_pre))
print('TP FP TN FN')
print('%d %d %d %d'%(tp,fp,tn,fn))
print('accuracy precision recall F1')
print('%.4lf %.4lf %.4lf %.4lf'%(accuracy, precision, recall, F1))
print('auc rmse')
print('%.4lf %.4lf'%(rfc_auc, rfc_rmse))

TP FP TN FN
73 24 144 27
accuracy precision recall F1
0.8097 0.7526 0.7300 0.7411
auc rmse
0.7936 0.4362
